In [1]:
import os
from PIL import Image
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
# def check_and_clean_images(folder_path):
#     for folder_name in os.listdir(folder_path):
#         folder_dir = os.path.join(folder_path, folder_name)
#         for img_file in os.listdir(folder_dir):
#             img_path = os.path.join(folder_dir, img_file)
#             try:
#                 # Muat gambar menggunakan PIL untuk memverifikasi keabsahannya
#                 with Image.open(img_path) as img:
#                     img.verify()
#                 # Memuat gambar menggunakan TensorFlow untuk verifikasi tambahan
#                 img_tensor = tf.io.read_file(img_path)
#                 img_tensor = tf.image.decode_image(img_tensor)
#             except (IOError, SyntaxError, tf.errors.InvalidArgumentError) as e:
#                 print(f"Corrupt or unsupported image removed: {img_path}")
#                 os.remove(img_path)

# dataset_path = "fresh-rotten/Train"
# check_and_clean_images(dataset_path)

In [3]:
image_size = (256, 256)
batch_size = 32

In [4]:
train_path = "fresh-rotten/Train"
val_path = "fresh-rotten/Validation"
test_path = "fresh-rotten/Test"

In [5]:
train_dataset = image_dataset_from_directory(
    train_path,
    label_mode="int",         
    image_size=image_size,
    batch_size=batch_size,
)
# .map(lambda x: (x, custom_label_mapping(x.file_path)))

val_dataset = image_dataset_from_directory(
    val_path,
    label_mode="int",
    image_size=image_size,
    batch_size=batch_size,
)
# .map(lambda x: (x, custom_label_mapping(x.file_path)))

test_dataset = image_dataset_from_directory(
    test_path,
    label_mode="int",
    image_size=image_size,
    batch_size=batch_size,
)
# .map(lambda x: (x, custom_label_mapping(x.file_path)))

Found 33412 files belonging to 32 classes.
Found 7127 files belonging to 32 classes.
Found 7283 files belonging to 32 classes.


In [6]:
def custom_label_mapping(image, label):
    new_label = tf.where(label < 9, 0, 1)
    return image, new_label

train_dataset = train_dataset.map(custom_label_mapping)
val_dataset = val_dataset.map(custom_label_mapping)
test_dataset = test_dataset.map(custom_label_mapping)

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

In [8]:
def process_image(image, label):
    # image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) / 255.0  
    return image, label

train_dataset = train_dataset.map(lambda x, y : (data_augmentation(x, training=True), y)).map(process_image)
val_dataset = val_dataset.map(process_image)
test_dataset = test_dataset.map(process_image)

In [9]:
def create_model():
    model = models.Sequential([
        layers.Input(shape=(256, 256, 3)),
        layers.Rescaling(1/255.),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [10]:
model = create_model()

In [11]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1392s 1s/step - accuracy: 0.8363 - loss: 0.4072 - val_accuracy: 0.9122 - val_loss: 0.2212
Epoch 2/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1334s 1s/step - accuracy: 0.8938 - loss: 0.2395 - val_accuracy: 0.9277 - val_loss: 0.1718
Epoch 3/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1874s 2s/step - accuracy: 0.9123 - loss: 0.2034 - val_accuracy: 0.8941 - val_loss: 0.2977
Epoch 4/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1351s 1s/step - accuracy: 0.9119 - loss: 0.2118 - val_accuracy: 0.9454 - val_loss: 0.1285
Epoch 5/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 2474s 2s/step - accuracy: 0.9285 - loss: 0.1655 - val_accuracy: 0.9341 - val_loss: 0.1587
Epoch 6/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1349s 1s/step - accuracy: 0.9388 - loss: 0.1502 - val_accuracy: 0.9530 - val_loss: 0.1187
Epoch 7/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 1330s 1s/step - accuracy: 0.9461 - loss: 0.1295 - val_accuracy: 0.9503 - val_loss: 0.1320
Epoch 8/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 2017s 2s/step - accuracy: 0.9555 -

In [13]:
val_loss, val_acc = model.evaluate(val_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)

print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

223/223 ━━━━━━━━━━━━━━━━━━━━ 92s 411ms/step - accuracy: 0.9674 - loss: 0.0815
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 494s 472ms/step - accuracy: 0.9645 - loss: 0.0847
228/228 ━━━━━━━━━━━━━━━━━━━━ 98s 422ms/step - accuracy: 0.9496 - loss: 0.1198
Train Accuracy		: 96.72% | Train Loss		: 0.08169645071029663
Validation Accuracy	: 96.60% | Validation Loss	: 0.08388444781303406
Test Accuracy		: 94.92% | Test Loss 		: 0.12273943424224854


In [14]:
model.save("fresh-rotten.h5")

In [12]:
model.save("fresh-rotten.keras")